In [0]:
!pip install requests

In [0]:
import requests
from pyspark.sql.types import StructType, StructField, StringType, DateType, TimestampType
from pyspark.sql import Row

In [0]:
# Run code from package definition
%run /Path/To/Your/Package # being Package your .py file

### PAT creation

In [0]:
# API call
workspace_url = '' # "https://<xxx-xxxxxxxx-xxxx>.cloud.databricks.com"
token = dbutils.secrets.get(scope="<scope_name>", key="<key_name>")
payload = {"comment": "Recreated token"}

token_value, token_id, expiry_timestamp = dbx_api.create_token(workspace_url, token, payload)

### PUT token as Secret in Scope

In [0]:
# API call
payload = {"scope": "<scope_name>", "key": "<key_name>", "string_value": token_value}

dbx_api.put_secret(workspace_url, token_value, payload)

### Update Job Schedule Refresh based on new PAT's expiry date

In [0]:
# API call
regenerate_timestamp = expiry_timestamp - timedelta(days=1)
updated_quartz_cron = f"0 {regenerate_timestamp.minute} {regenerate_timestamp.hour} {regenerate_timestamp.day} {regenerate_timestamp.month} ? {regenerate_timestamp.year}"
payload = {"job_id": 111111111111111, # job id as integer
           "new_settings": {"schedule": {"quartz_cron_expression": updated_quartz_cron, "timezone_id": "America/Buenos_Aires"}
                            }
           }

dbx_api.update_job(workspace_url, token_value, payload)

### Alert on expiry date

In [0]:
data = [Row(asset="<token_name>", expiry_timestamp=expiry_timestamp, alert_date=regenerate_timestamp.date(), alert_timestamp=regenerate_timestamp)]
schema = StructType(
            [StructField("asset", StringType()),
             StructField("expiry_timestamp", TimestampType()),
             StructField("alert_date", DateType()),
             StructField("alert_timestamp", TimestampType())
            ])
df_alert = spark.createDataFrame(data, schema)
df_alert.createOrReplaceTempView("df_alert")

In [0]:
# Overwrite alerts table
df_alert.write.format("delta") \
              .mode("overwrite") \
              .option("mergeSchema", "true") \
              .saveAsTable("<catalog>.<schema>.alerts")

In [0]:
# API call
# updates alert trigger only on new regeneration date
alert_id = "" # retrieve it from its URL
alert_quartz_cron = f"0 0 0 {regenerate_timestamp.day} {regenerate_timestamp.month} ? {regenerate_timestamp.year}"
payload = {"schedule": {"pause_status": "UNPAUSED", "quartz_cron_schedule": alert_quartz_cron, "timezone_id": "America/Buenos_Aires"
                        }
           }

update_alert(workspace_url, alert_id, token_value, payload)

-- Query for Alert
SELECT *
FROM catalog.schema.alerts
WHERE alert_date = current_date()